In [1]:
!pip install transformers
!pip install sentencepiece
!pip install openpyxl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 4.5 MB/s 
     |████████████████████████████████| 182 kB 70.6 MB/s 
     |████████████████████████████████| 7.6 MB 70.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 4.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import sys
import os
import time
import re
import random
from typing import Dict, List, Optional, Union
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc

from google.colab import files

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import tensorflow as tf
from transformers import BertTokenizer, BertConfig, TFBertForSequenceClassification
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification
from transformers import ElectraTokenizer, TFElectraForSequenceClassification
from transformers import XLNetTokenizer, TFXLNetForSequenceClassification
from transformers import LongformerTokenizer, TFLongformerForSequenceClassification
from transformers import DebertaTokenizer, TFDebertaForSequenceClassification

In [3]:
# set seed, TF uses python ramdom and numpy library, so these must also be fixed
tf.random.set_seed(0)
random.seed(0)
np.random.seed(0)
os.environ['PYTHONHASHSEED']=str(0)
os.environ['TF_DETERMINISTIC_OPS'] = '0'

In [4]:
PATH_sg1 = "/content/drive/MyDrive/Colab Notebooks/data/final_labels_SG1.xlsx"
PATH_sg2 = "/content/drive/MyDrive/Colab Notebooks/data/final_labels_SG2.xlsx"
df_sg1 = pd.read_excel(PATH_sg1)
df_sg2 = pd.read_excel(PATH_sg2)
df_sg1.rename(columns={'text': 'sentence', 'label_bias': 'Label_bias'}, inplace=True)
df_sg2.rename(columns={'text': 'sentence', 'label_bias': 'Label_bias'}, inplace=True)
df_sg1.head()

,sentence,news_link,outlet,topic,type,Label_bias,label_opinion,biased_words
0,The Republican president assumed he was helpin...,http://www.msnbc.com/rachel-maddow-show/auto-i...,msnbc,environment,left,Biased,Expresses writer’s opinion,[]
1,Though the indictment of a woman for her own p...,https://eu.usatoday.com/story/news/nation/2019...,usa-today,abortion,center,Non-biased,Somewhat factual but also opinionated,[]
2,Ingraham began the exchange by noting American...,https://www.breitbart.com/economy/2020/01/12/d...,breitbart,immigration,right,No agreement,No agreement,['flood']
3,The tragedy of America’s 18 years in Afghanist...,http://feedproxy.google.com/~r/breitbart/~3/ER...,breitbart,international-politics-and-world-news,right,Biased,Somewhat factual but also opinionated,"['tragedy', 'stubborn']"
4,The justices threw out a challenge from gun ri...,https://www.huffpost.com/entry/supreme-court-g...,msnbc,gun-control,left,Non-biased,Entirely factual,[]


In [5]:
# binarize classification problem
df_sg1 = df_sg1[df_sg1['Label_bias']!='No agreement']
df_sg1 = df_sg1[df_sg1['Label_bias'].isna()==False]
df_sg1.replace(to_replace='Biased', value=1, inplace=True)
df_sg1.replace(to_replace='Non-biased', value=0, inplace=True)

df_sg2 = df_sg2[df_sg2['Label_bias']!='No agreement']
df_sg2.replace(to_replace='Biased', value=1, inplace=True)
df_sg2.replace(to_replace='Non-biased', value=0, inplace=True)

# # test pipeline set
# df_sg1, exclude = train_test_split(df_sg1, test_size=0.95)
# df_sg2, exclude = train_test_split(df_sg2, test_size=0.8)

In [51]:
# Stratified k-Fold instance
skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [6]:
# helper functions called in skfold loop

def pd_to_tf(df):
    """convert a pandas dataframe into a tensorflow dataset"""
    target = df.pop('Label_bias')
    sentence = df.pop('sentence')
    return tf.data.Dataset.from_tensor_slices((sentence.values, target.values))

def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history['val_'+metric], '')
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend([metric, 'val_'+metric])
    plt.show()

def tokenize(df, model_name):
    """convert a pandas dataframe into a tensorflow dataset and run hugging face's tokenizer on data"""
    df2 = df.copy(deep=False)
    target = df2.pop('Label_bias')
    sentence = df2.pop('sentence')
    
    if model_name=='bert':
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    elif model_name=='roberta':
        tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
    elif model_name=='deberta':
        tokenizer = DebertaTokenizer.from_pretrained("kamalkraj/deberta-base")
    elif model_name=='electra':
        tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-discriminator')

    train_encodings = tokenizer(
                        sentence.tolist(),                      
                        add_special_tokens = True, # add [CLS], [SEP]
                        truncation = True, # cut off at max length of the text that can go to BERT
                        padding = True, # add [PAD] tokens
                        return_attention_mask = True, # add attention mask to not focus on pad tokens
              )
    
    dataset = tf.data.Dataset.from_tensor_slices(
        (dict(train_encodings), 
         target.tolist()))
    
    # clear unused memory
    del(df2)
    del(target)
    del(sentence)
    del(tokenizer)
    del(train_encodings)
    gc.collect()
    
    return dataset

In [48]:
def run_model_5fold(df_name, df_train, model_name, freeze_encoder=False, pretrained=False, plot=False, batch_size=32, epochs=10):
    """"function to run 5-fold cross validation for any provided model"""

    # these variables will be needed for skfold to select indices
    Y = df_train['Label_bias']
    X = df_train['sentence']

    # hyperparams
    BUFFER_SIZE = 10000
    BATCH_SIZE = batch_size
    k = 1

    val_loss = []
    val_acc = []
    val_prec = []
    val_rec = []
    val_f1 = []
    val_f1_micro = []
    val_f1_wmacro = []
    
    if pretrained==True:
        optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
        
        if model_name=='bert':
            transfer_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')
            
        elif model_name=='roberta':
            transfer_model = TFRobertaForSequenceClassification.from_pretrained('roberta-base')
            
        elif model_name=='deberta':
            transfer_model = TFDebertaForSequenceClassification.from_pretrained("kamalkraj/deberta-base")
            
        transfer_model.compile(optimizer=optimizer, loss='binary_crossentropy') 
        transfer_model.load_weights(f'/content/drive/MyDrive/Colab Notebooks/weights/{model_name}_final_checkpoint_news_headlines_USA')
        trained_model_layer = transfer_model.get_layer(index=0).get_weights()
            

    for train_index, val_index in skfold.split(X,Y):
        print('### Start fold {}'.format(k))

        # split into train and validation set
        train_dataset = df_train.iloc[train_index]
        val_dataset = df_train.iloc[val_index]

        # prepare data for transformer
        train_dataset = tokenize(train_dataset, model_name)
        val_dataset = tokenize(val_dataset, model_name)

        # mini-batch it
        train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
        val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

        # create new model
        if model_name == 'bert':
            model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
        elif model_name == 'roberta':
            model = TFRobertaForSequenceClassification.from_pretrained('roberta-base')
        elif model_name == 'electra':
            model = TFElectraForSequenceClassification.from_pretrained('google/electra-small-discriminator')
        elif model_name == 'deberta':
            model = TFDebertaForSequenceClassification.from_pretrained("kamalkraj/deberta-base")

        # freeze flags whether encoder layer should be frozen to not destroy transfer learning. Only set to false when enough data is provided
        if freeze_encoder == True:
            for w in model.get_layer(index=0).weights:
                w._trainable = False

        # compile it
        optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5) 
        model.compile(optimizer=optimizer, loss='binary_crossentropy') 

        # transfer learning
        if pretrained == True:
            model.get_layer(index=0).set_weights(trained_model_layer) # load bias-specific weights

        # after 2 epochs without improvement, stop training
        callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

        # fit it
        history = model.fit(train_dataset, epochs=epochs, validation_data = val_dataset, callbacks=[callback])

        # plot history
        if plot:
            plot_graphs(history,'loss')

        # evaluate
        loss = model.evaluate(val_dataset)

        logits = model.predict(val_dataset)  
        yhats = []
        for i in logits[0]:
            # assign class label according to highest logit
            candidates = i.tolist()
            decision = candidates.index(max(candidates))
            yhats.append(decision)

        y = []
        for text, label in val_dataset.unbatch():   
            y.append(label.numpy())
        

        val_loss.append(loss)
        val_acc.append(accuracy_score(y, yhats))
        val_prec.append(precision_score(y, yhats))
        val_rec.append(recall_score(y, yhats))
        val_f1.append(f1_score(y, yhats))
        val_f1_micro.append(f1_score(y, yhats, average='micro'))
        val_f1_wmacro.append(f1_score(y, yhats, average='weighted'))
        
        

        tf.keras.backend.clear_session()
        
        # clear unused memory
        del(train_dataset)
        del(val_dataset)
        del(history)
        del(optimizer)
        del(callback)
        del(loss)
        del(y)
        gc.collect()

        k += 1
    
    # save model weights
    trained_layer = model.get_layer(index=0).get_weights()
    model.save_weights(f'/content/drive/MyDrive/Colab Notebooks/weights/{model_name}_{pretrained}_{df_name}_main')

    return {'loss': val_loss, 'acc': val_acc, 'prec': val_prec, 'rec': val_rec, 'f1': val_f1, 
            'f1_micro': val_f1_micro, 'f1_wmacro': val_f1_wmacro, 'model_name': model_name, 
            'distant': pretrained, 'df_name': df_name} 

In [46]:
def measure(d, results):
    loss_cv = np.mean(d['loss'])
    acc_cv = np.mean(d['acc'])
    prec_cv = np.mean(d['prec'])
    rec_cv = np.mean(d['rec'])
    f1_cv = np.mean(d['f1'])
    f1_micro_cv = np.mean(d['f1_micro'])
    f1_wmacro_cv = np.mean(d['f1_wmacro'])
    
    row = {
        'Dataset': d['df_name'],
        'Model': d['model_name'], 
        'Distant': d['distant'], 
        'Loss': loss_cv, 
        'Accuracy': acc_cv, 
        'Precision': prec_cv, 
        'Recall': rec_cv, 
        'F1': f1_cv, 
        'F1 Micro': f1_micro_cv, 
        'F1 Weighted': f1_wmacro_cv
    }
    
    results = results.append(row, ignore_index=True)
    
    return results

In [47]:
## instantiate results df
columns = ['Dataset', 'Model', 'Distant', 'Loss', 'Accuracy', 'Precision', 'Recall', 'F1', 'F1 Micro', 'F1 Weighted']
results = pd.DataFrame(columns=columns)

# BERT

In [ ]:
# Sg1 
results = measure(run_model_5fold('sg1', df_sg1, 'bert', freeze_encoder=False, pretrained=False, plot=False), results)
print(results)

### Start fold 1


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
39/39 [==============================] - 52s 908ms/step - loss: 0.6820 - val_loss: 0.6231
Epoch 2/10
39/39 [==============================] - 31s 804ms/step - loss: 0.6370 - val_loss: 0.5427
Epoch 3/10
39/39 [==============================] - 31s 801ms/step - loss: 0.4794 - val_loss: 0.7569
Epoch 4/10
10/10 [==============================] - 5s 195ms/step
### Start fold 2


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
39/39 [==============================] - 51s 920ms/step - loss: 1.0844 - val_loss: 0.6939
Epoch 2/10
39/39 [==============================] - 32s 833ms/step - loss: 0.6492 - val_loss: 0.6145
Epoch 3/10
39/39 [==============================] - 32s 822ms/step - loss: 0.8369 - val_loss: 0.7789
Epoch 4/10
10/10 [==============================] - 6s 250ms/step
### Start fold 3


...

Epoch 1/10
39/39 [==============================] - 50s 895ms/step - loss: 0.6915 - val_loss: 0.6266
Epoch 2/10
39/39 [==============================] - 32s 808ms/step - loss: 0.6123 - val_loss: 0.5787
Epoch 3/10
39/39 [==============================] - 31s 793ms/step - loss: 0.3721 - val_loss: 1.8531
Epoch 4/10
10/10 [==============================] - 6s 190ms/step
### Start fold 4


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
39/39 [==============================] - 50s 898ms/step - loss: 0.7468 - val_loss: 0.8687
Epoch 2/10
39/39 [==============================] - 32s 814ms/step - loss: 0.6962 - val_loss: 0.6147
Epoch 3/10
39/39 [==============================] - 32s 812ms/step - loss: 0.5223 - val_loss: 0.5228
Epoch 4/10
39/39 [==============================] - 31s 805ms/step - loss: 0.6727 - val_loss: 0.6500
Epoch 5/10
10/10 [==============================] - 6s 208ms/step
### Start fold 5


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
39/39 [==============================] - 51s 916ms/step - loss: 1.8748 - val_loss: 0.6302
Epoch 2/10
39/39 [==============================] - 32s 823ms/step - loss: 0.6072 - val_loss: 0.5268
Epoch 3/10
39/39 [==============================] - 32s 821ms/step - loss: 0.5049 - val_loss: 0.5215
Epoch 4/10
39/39 [==============================] - 31s 800ms/step - loss: 0.3730 - val_loss: 1.2530
Epoch 5/10
10/10 [==============================] - 6s 249ms/step
  Dataset Model Distant     Loss  Accuracy  Precision    Recall        F1  \
0     sg1  bert   False  0.55603  0.451406   0.400755  0.487284  0.420398   

   F1 Micro  F1 Weighted  
0  0.451406     0.413136  


In [ ]:
# Sg2 
results = measure(run_model_5fold('sg2', df_sg2, 'bert', freeze_encoder=False, pretrained=False, plot=False), results)
print(results)

### Start fold 1


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
92/92 [==============================] - 103s 851ms/step - loss: 0.8114 - val_loss: 0.5954
Epoch 2/10
92/92 [==============================] - 74s 807ms/step - loss: 0.5850 - val_loss: 0.5808
Epoch 3/10
92/92 [==============================] - 74s 804ms/step - loss: 0.5578 - val_loss: 1.9449
Epoch 4/10
23/23 [==============================] - 8s 210ms/step
### Start fold 2


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
92/92 [==============================] - 94s 853ms/step - loss: 4.1306 - val_loss: 4.0944
Epoch 2/10
92/92 [==============================] - 75s 811ms/step - loss: 4.0864 - val_loss: 4.0666
Epoch 3/10
92/92 [==============================] - 75s 810ms/step - loss: 4.0769 - val_loss: 4.0642
Epoch 4/10
92/92 [==============================] - 72s 782ms/step - loss: 6.6546 - val_loss: 7.5970
Epoch 5/10
23/23 [==============================] - 9s 228ms/step


...

Epoch 1/10
92/92 [==============================] - 94s 849ms/step - loss: 0.7139 - val_loss: 0.5838
Epoch 2/10
92/92 [==============================] - 75s 811ms/step - loss: 0.7023 - val_loss: 0.5615
Epoch 3/10
92/92 [==============================] - 74s 802ms/step - loss: 0.4800 - val_loss: 0.6918
Epoch 4/10
23/23 [==============================] - 8s 212ms/step
### Start fold 4


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
92/92 [==============================] - 94s 857ms/step - loss: 0.9733 - val_loss: 0.6904
Epoch 2/10
92/92 [==============================] - 75s 815ms/step - loss: 0.7640 - val_loss: 0.8117
Epoch 3/10
92/92 [==============================] - 75s 815ms/step - loss: 0.7932 - val_loss: 0.6873
Epoch 4/10
92/92 [==============================] - 75s 817ms/step - loss: 0.7252 - val_loss: 0.6248
Epoch 5/10
92/92 [==============================] - 75s 818ms/step - loss: 0.6816 - val_loss: 0.5944
Epoch 6/10
92/92 [==============================] - 75s 815ms/step - loss: 0.5892 - val_loss: 0.7159
Epoch 7/10
23/23 [==============================] - 9s 238ms/step
### Start fold 5


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
92/92 [==============================] - 90s 809ms/step - loss: 0.8279 - val_loss: 0.5310
Epoch 2/10
92/92 [==============================] - 71s 767ms/step - loss: 0.7823 - val_loss: 0.5282
Epoch 3/10
92/92 [==============================] - 71s 768ms/step - loss: 0.5862 - val_loss: 0.5254
Epoch 4/10
92/92 [==============================] - 70s 758ms/step - loss: 0.4830 - val_loss: 0.7231
Epoch 5/10
23/23 [==============================] - 10s 258ms/step
  Dataset Model Distant      Loss  Accuracy  Precision    Recall        F1  \
0     sg1  bert   False  0.556030  0.451406   0.400755  0.487284  0.420398   
1     sg2  bert   False  1.265255  0.520005   0.428627  0.239779  0.198883   

   F1 Micro  F1 Weighted  
0  0.451406     0.413136  
1  0.520005     0.378480  


# RoBERTa

In [ ]:
# Sg1 
results = measure(run_model_5fold('sg1', df_sg1, 'roberta', freeze_encoder=False, pretrained=False, plot=False), results)
print(results)

### Start fold 1


Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/657M [00:00<?, ?B/s]

...

Epoch 1/10
39/39 [==============================] - 25s 207ms/step - loss: 1.3668 - val_loss: 0.8177
Epoch 2/10
39/39 [==============================] - 5s 121ms/step - loss: 0.7565 - val_loss: 0.6786
Epoch 3/10
39/39 [==============================] - 4s 113ms/step - loss: 0.6758 - val_loss: 0.6998
Epoch 4/10
10/10 [==============================] - 3s 38ms/step
### Start fold 4


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
39/39 [==============================] - 25s 207ms/step - loss: 1.1117 - val_loss: 0.5658
Epoch 2/10
39/39 [==============================] - 5s 121ms/step - loss: 0.7099 - val_loss: 0.5633
Epoch 3/10
39/39 [==============================] - 4s 112ms/step - loss: 0.5624 - val_loss: 0.6306
Epoch 4/10
10/10 [==============================] - 3s 36ms/step
### Start fold 5


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
39/39 [==============================] - 25s 209ms/step - loss: 0.8567 - val_loss: 0.5624
Epoch 2/10
39/39 [==============================] - 4s 115ms/step - loss: 0.7175 - val_loss: 0.6120
Epoch 3/10
39/39 [==============================] - 5s 123ms/step - loss: 0.6069 - val_loss: 0.4838
Epoch 4/10
39/39 [==============================] - 4s 114ms/step - loss: 0.5030 - val_loss: 0.5995
Epoch 5/10
39/39 [==============================] - 5s 122ms/step - loss: 0.4546 - val_loss: 0.4457
Epoch 6/10
39/39 [==============================] - 4s 115ms/step - loss: 0.4707 - val_loss: 1.4829
Epoch 7/10
10/10 [==============================] - 3s 43ms/step
  Dataset    Model Distant      Loss  Accuracy  Precision    Recall        F1  \
0     sg1  roberta   False  0.580312  0.515538    0.56224  0.636251  0.460742   

   F1 Micro  F1 Weighted  
0  0.515538     0.377869  


In [ ]:
# Sg2 
results = measure(run_model_5fold('sg2', df_sg2, 'roberta', freeze_encoder=False, pretrained=False, plot=False), results)
print(results)

### Start fold 1


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
92/92 [==============================] - 31s 154ms/step - loss: 0.7759 - val_loss: 0.5938
Epoch 2/10
92/92 [==============================] - 11s 117ms/step - loss: 0.6193 - val_loss: 0.5434
Epoch 3/10
92/92 [==============================] - 11s 117ms/step - loss: 0.5085 - val_loss: 0.5409
Epoch 4/10
92/92 [==============================] - 10s 114ms/step - loss: 0.5039 - val_loss: 0.6502
Epoch 5/10
23/23 [==============================] - 4s 40ms/step
### Start fold 2


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
92/92 [==============================] - 31s 153ms/step - loss: 0.7901 - val_loss: 0.7018
Epoch 2/10
92/92 [==============================] - 11s 117ms/step - loss: 0.7182 - val_loss: 0.6636
Epoch 3/10
92/92 [==============================] - 10s 114ms/step - loss: 0.7071 - val_loss: 0.7612
Epoch 4/10
92/92 [==============================] - 11s 117ms/step - loss: 0.6478 - val_loss: 0.6100
Epoch 5/10
92/92 [==============================] - 10s 113ms/step - loss: 0.5719 - val_loss: 0.6369
Epoch 6/10
92/92 [==============================] - 11s 117ms/step - loss: 0.5555 - val_loss: 0.6095
Epoch 7/10
92/92 [==============================] - 11s 117ms/step - loss: 0.5858 - val_loss: 0.5945
Epoch 8/10
92/92 [==============================] - 10s 113ms/step - loss: 0.5188 - val_loss: 0.6091
Epoch 9/10
23/23 [==============================] - 4s 37ms/step


...

Epoch 1/10
92/92 [==============================] - 31s 153ms/step - loss: 0.8103 - val_loss: 0.5426
Epoch 2/10
92/92 [==============================] - 10s 113ms/step - loss: 0.6750 - val_loss: 0.5429
Epoch 3/10
92/92 [==============================] - 11s 117ms/step - loss: 0.5966 - val_loss: 0.4643
Epoch 4/10
92/92 [==============================] - 10s 113ms/step - loss: 0.7581 - val_loss: 0.6529
Epoch 5/10
23/23 [==============================] - 4s 37ms/step
### Start fold 4


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
92/92 [==============================] - 31s 153ms/step - loss: 0.7457 - val_loss: 0.6952
Epoch 2/10
92/92 [==============================] - 11s 118ms/step - loss: 0.6403 - val_loss: 0.6022
Epoch 3/10
92/92 [==============================] - 11s 117ms/step - loss: 0.5368 - val_loss: 0.5399
Epoch 4/10
92/92 [==============================] - 11s 117ms/step - loss: 0.5920 - val_loss: 0.4837
Epoch 5/10
92/92 [==============================] - 10s 114ms/step - loss: 0.5621 - val_loss: 0.5652
Epoch 6/10
23/23 [==============================] - 4s 38ms/step
### Start fold 5


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
92/92 [==============================] - 30s 142ms/step - loss: 0.7377 - val_loss: 0.4795
Epoch 2/10
92/92 [==============================] - 9s 102ms/step - loss: 0.7714 - val_loss: 0.6936
Epoch 3/10
23/23 [==============================] - 4s 41ms/step
  Dataset    Model Distant      Loss  Accuracy  Precision    Recall        F1  \
0     sg1  roberta   False  0.580312  0.515538   0.562240  0.636251  0.460742   
1     sg2  roberta   False  0.512574  0.545579   0.522718  0.595028  0.418805   

   F1 Micro  F1 Weighted  
0  0.515538     0.377869  
1  0.545579     0.419941  


# DeBERTa

In [ ]:
# Sg1 
results = measure(run_model_5fold('sg1', df_sg1, 'deberta', freeze_encoder=False, pretrained=False, plot=False), results)
print(results)

### Start fold 1


Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/744 [00:00<?, ?B/s]

...

Epoch 1/10
39/39 [==============================] - 56s 506ms/step - loss: 2.0577 - val_loss: 0.6662
Epoch 2/10
39/39 [==============================] - 12s 319ms/step - loss: 0.7593 - val_loss: 0.8156
Epoch 3/10
10/10 [==============================] - 7s 43ms/step
### Start fold 5


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
All model checkpoint layers were used when initializing TFDebertaForSequenceClassification.

Some layers of TFDebertaForSequenceClassification were not initialized from the model checkpoint at kamalkraj/deberta-base and are newly initialized: ['classifier', 'cls_dropout', 'pooler']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
39/39 [==============================] - 56s 506ms/step - loss: 3.9307 - val_loss: 4.0217
Epoch 2/10
39/39 [==============================] - 12s 315ms/step - loss: 4.0521 - val_loss: 4.6274
Epoch 3/10
39/39 [==============================] - 13s 322ms/step - loss: 4.1095 - val_loss: 4.0139
Epoch 4/10
39/39 [==============================] - 12s 313ms/step - loss: 4.0296 - val_loss: 4.0262
Epoch 5/10
10/10 [==============================] - 7s 53ms/step


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


  Dataset    Model Distant      Loss  Accuracy  Precision    Recall        F1  \
0     sg1  roberta   False  0.580312  0.515538   0.562240  0.636251  0.460742   
1     sg2  roberta   False  0.512574  0.545579   0.522718  0.595028  0.418805   
2     sg1  deberta   False  1.786612  0.513528   0.431187  0.633163  0.467586   

   F1 Micro  F1 Weighted  
0  0.515538     0.377869  
1  0.545579     0.419941  
2  0.513528     0.389934  


In [ ]:
# Sg2
results = measure(run_model_5fold('sg2', df_sg2, 'deberta', freeze_encoder=False, pretrained=False, plot=False), results)
print(results)

### Start fold 1


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
All model checkpoint layers were used when initializing TFDebertaForSequenceClassification.

Some layers of TFDebertaForSequenceClassification were not initialized from the model checkpoint at kamalkraj/deberta-base and are newly initialized: ['classifier', 'cls_dropout', 'pooler']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
92/92 [==============================] - 72s 398ms/step - loss: 2.5788 - val_loss: 4.1659
Epoch 2/10
92/92 [==============================] - 30s 321ms/step - loss: 5.0330 - val_loss: 7.6679
Epoch 3/10
23/23 [==============================] - 8s 47ms/step
### Start fold 2


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
All model checkpoint layers were used when initializing TFDebertaForSequenceClassification.

Some layers of TFDebertaForSequenceClassification were not initialized from the model checkpoint at kamalkraj/deberta-base and are newly initialized: ['classifier', 'cls_dropout', 'pooler']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
92/92 [==============================] - 73s 400ms/step - loss: 4.0199 - val_loss: 4.0762
Epoch 2/10
92/92 [==============================] - 30s 326ms/step - loss: 4.1358 - val_loss: 4.0649
Epoch 3/10
92/92 [==============================] - 30s 321ms/step - loss: 4.1109 - val_loss: 4.3380
Epoch 4/10
23/23 [==============================] - 7s 44ms/step


...

Epoch 1/10
92/92 [==============================] - 73s 400ms/step - loss: 0.9726 - val_loss: 0.6936
Epoch 2/10
92/92 [==============================] - 30s 321ms/step - loss: 3.0709 - val_loss: 4.1147
Epoch 3/10
23/23 [==============================] - 7s 46ms/step
### Start fold 5


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
All model checkpoint layers were used when initializing TFDebertaForSequenceClassification.

Some layers of TFDebertaForSequenceClassification were not initialized from the model checkpoint at kamalkraj/deberta-base and are newly initialized: ['classifier', 'cls_dropout', 'pooler']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
92/92 [==============================] - 69s 355ms/step - loss: 4.7392 - val_loss: 5.8658
Epoch 2/10
92/92 [==============================] - 26s 280ms/step - loss: 4.2130 - val_loss: 4.0721
Epoch 3/10
92/92 [==============================] - 26s 279ms/step - loss: 4.1448 - val_loss: 4.0514
Epoch 4/10
92/92 [==============================] - 26s 279ms/step - loss: 3.7108 - val_loss: 0.7227
Epoch 5/10
92/92 [==============================] - 25s 275ms/step - loss: 1.0382 - val_loss: 0.7329
Epoch 6/10
92/92 [==============================] - 26s 279ms/step - loss: 0.7069 - val_loss: 0.6945
Epoch 7/10
92/92 [==============================] - 25s 275ms/step - loss: 0.6991 - val_loss: 0.6966
Epoch 8/10
92/92 [==============================] - 26s 279ms/step - loss: 0.6993 - val_loss: 0.6931
Epoch 9/10
92/92 [==============================] - 25s 275ms/step - loss: 0.7037 - val_loss: 0.6931
Epoch 10/10
23/23 [==============================] - 8s 53ms/step


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


  Dataset    Model Distant      Loss  Accuracy  Precision    Recall        F1  \
0     sg1  roberta   False  0.580312  0.515538   0.562240  0.636251  0.460742   
1     sg2  roberta   False  0.512574  0.545579   0.522718  0.595028  0.418805   
2     sg1  deberta   False  1.786612  0.513528   0.431187  0.633163  0.467586   
3     sg2  deberta   False  2.062078  0.500138   0.296457  0.600000  0.396837   

   F1 Micro  F1 Weighted  
0  0.515538     0.377869  
1  0.545579     0.419941  
2  0.513528     0.389934  
3  0.500138     0.335278  


# BERT w/ Distant

In [ ]:
# Sg1 
results = measure(run_model_5fold('sg1', df_sg1, 'bert', freeze_encoder=False, pretrained=True, plot=False), results)
print(results)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Start fold 1


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
39/39 [==============================] - 24s 204ms/step - loss: 2.4129 - val_loss: 0.6518
Epoch 2/10
39/39 [==============================] - 5s 118ms/step - loss: 0.5884 - val_loss: 0.5706
Epoch 3/10
39/39 [==============================] - 4s 109ms/step - loss: 0.4911 - val_loss: 0.6041
Epoch 4/10
10/10 [==============================] - 4s 37ms/step
### Start fold 5


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
39/39 [==============================] - 24s 204ms/step - loss: 4.1121 - val_loss: 4.0460
Epoch 2/10
39/39 [==============================] - 5s 120ms/step - loss: 4.0494 - val_loss: 3.9925
Epoch 3/10
39/39 [==============================] - 4s 111ms/step - loss: 3.9971 - val_loss: 4.0068
Epoch 4/10
10/10 [==============================] - 4s 42ms/step


  Dataset    Model Distant      Loss  Accuracy  Precision    Recall        F1  \
0     sg1  roberta   False  0.580312  0.515538   0.562240  0.636251  0.460742   
1     sg2  roberta   False  0.512574  0.545579   0.522718  0.595028  0.418805   
2     sg1  deberta   False  1.786612  0.513528   0.431187  0.633163  0.467586   
3     sg2  deberta   False  2.062078  0.500138   0.296457  0.600000  0.396837   
4     sg1  electra   False  0.550895  0.475357   0.444049  0.380805  0.393093   
5     sg2  electra   False  0.482666  0.506686   0.651247  0.436464  0.394047   
6     sg1  deberta    True  0.617424  0.492855   0.334588  0.584009  0.418621   
7     sg2  deberta    True  0.620772  0.412249   0.402475  0.313812  0.241562   
8     sg1     bert    True  1.253572  0.452740   0.246955  0.457718  0.316711   

   F1 Micro  F1 Weighted  
0  0.515538     0.377869  
1  0.545579     0.419941  
2  0.513528     0.389934  
3  0.500138     0.335278  
4  0.475357     0.448213  
5  0.506686     0.440985  


In [ ]:
# Sg2 
results = measure(run_model_5fold('sg2', df_sg2, 'bert', freeze_encoder=False, pretrained=True, plot=False), results)
print(results)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Start fold 1


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
92/92 [==============================] - 31s 150ms/step - loss: 1.1266 - val_loss: 0.6763
Epoch 2/10
92/92 [==============================] - 10s 114ms/step - loss: 0.6339 - val_loss: 0.5299
Epoch 3/10
92/92 [==============================] - 10s 109ms/step - loss: 0.5597 - val_loss: 0.6219
Epoch 4/10
23/23 [==============================] - 4s 39ms/step
### Start fold 2


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
92/92 [==============================] - 30s 149ms/step - loss: 0.7249 - val_loss: 0.5362
Epoch 2/10
92/92 [==============================] - 10s 110ms/step - loss: 0.5609 - val_loss: 0.9209
Epoch 3/10
23/23 [==============================] - 5s 40ms/step
### Start fold 5


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
92/92 [==============================] - 30s 143ms/step - loss: 2.0889 - val_loss: 0.6000
Epoch 2/10
92/92 [==============================] - 10s 107ms/step - loss: 0.5174 - val_loss: 0.5818
Epoch 3/10
92/92 [==============================] - 9s 103ms/step - loss: 0.4599 - val_loss: 1.8586
Epoch 4/10
23/23 [==============================] - 5s 41ms/step


  Dataset    Model Distant      Loss  Accuracy  Precision    Recall        F1  \
0     sg1  roberta   False  0.580312  0.515538   0.562240  0.636251  0.460742   
1     sg2  roberta   False  0.512574  0.545579   0.522718  0.595028  0.418805   
2     sg1  deberta   False  1.786612  0.513528   0.431187  0.633163  0.467586   
3     sg2  deberta   False  2.062078  0.500138   0.296457  0.600000  0.396837   
4     sg1  electra   False  0.550895  0.475357   0.444049  0.380805  0.393093   
5     sg2  electra   False  0.482666  0.506686   0.651247  0.436464  0.394047   
6     sg1  deberta    True  0.617424  0.492855   0.334588  0.584009  0.418621   
7     sg2  deberta    True  0.620772  0.412249   0.402475  0.313812  0.241562   
8     sg1     bert    True  1.253572  0.452740   0.246955  0.457718  0.316711   
9     sg2     bert    True  0.532223  0.558989   0.539072  0.743646  0.606827   

   F1 Micro  F1 Weighted  
0  0.515538     0.377869  
1  0.545579     0.419941  
2  0.513528     0.389934  


# RoBERTa w/ Distant

In [ ]:
# Sg1 
results = measure(run_model_5fold('sg1', df_sg1, 'roberta', freeze_encoder=False, pretrained=True, plot=False), results)
print(results)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Start fold 1


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
39/39 [==============================] - 25s 210ms/step - loss: 0.7682 - val_loss: 0.5338
Epoch 2/10
39/39 [==============================] - 5s 124ms/step - loss: 0.5815 - val_loss: 0.4845
Epoch 3/10
39/39 [==============================] - 4s 113ms/step - loss: 0.7735 - val_loss: 0.5143
Epoch 4/10
10/10 [==============================] - 3s 38ms/step
### Start fold 2


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
39/39 [==============================] - 25s 212ms/step - loss: 0.9474 - val_loss: 0.6919
Epoch 2/10
39/39 [==============================] - 5s 123ms/step - loss: 0.6788 - val_loss: 0.6088
Epoch 3/10
39/39 [==============================] - 4s 113ms/step - loss: 0.7213 - val_loss: 0.6967
Epoch 4/10
10/10 [==============================] - 4s 37ms/step
### Start fold 5


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
39/39 [==============================] - 25s 211ms/step - loss: 1.6020 - val_loss: 0.5812
Epoch 2/10
39/39 [==============================] - 4s 114ms/step - loss: 0.6505 - val_loss: 0.5891
Epoch 3/10
39/39 [==============================] - 5s 123ms/step - loss: 0.6213 - val_loss: 0.5082
Epoch 4/10
39/39 [==============================] - 5s 122ms/step - loss: 0.5875 - val_loss: 0.4699
Epoch 5/10
39/39 [==============================] - 4s 114ms/step - loss: 0.4281 - val_loss: 0.7031
Epoch 6/10
10/10 [==============================] - 4s 43ms/step


   Dataset    Model Distant      Loss  Accuracy  Precision    Recall  \
0      sg1  roberta   False  0.580312  0.515538   0.562240  0.636251   
1      sg2  roberta   False  0.512574  0.545579   0.522718  0.595028   
2      sg1  deberta   False  1.786612  0.513528   0.431187  0.633163   
3      sg2  deberta   False  2.062078  0.500138   0.296457  0.600000   
4      sg1  electra   False  0.550895  0.475357   0.444049  0.380805   
5      sg2  electra   False  0.482666  0.506686   0.651247  0.436464   
6      sg1  deberta    True  0.617424  0.492855   0.334588  0.584009   
7      sg2  deberta    True  0.620772  0.412249   0.402475  0.313812   
8      sg1     bert    True  1.253572  0.452740   0.246955  0.457718   
9      sg2     bert    True  0.532223  0.558989   0.539072  0.743646   
10     sg1  roberta    True  0.546565  0.468920   0.549649  0.386568   

          F1  F1 Micro  F1 Weighted  
0   0.460742  0.515538     0.377869  
1   0.418805  0.545579     0.419941  
2   0.467586  0.51352

In [ ]:
# Sg2
results = measure(run_model_5fold('sg2', df_sg2, 'roberta', freeze_encoder=False, pretrained=True, plot=False), results)
print(results)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Start fold 1


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
92/92 [==============================] - 32s 155ms/step - loss: 0.7223 - val_loss: 0.4452
Epoch 2/10
92/92 [==============================] - 11s 115ms/step - loss: 0.5301 - val_loss: 0.4863
Epoch 3/10
23/23 [==============================] - 4s 38ms/step
### Start fold 2


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
92/92 [==============================] - 31s 155ms/step - loss: 1.2645 - val_loss: 0.6738
Epoch 2/10
92/92 [==============================] - 11s 118ms/step - loss: 0.6424 - val_loss: 0.5209
Epoch 3/10
92/92 [==============================] - 10s 114ms/step - loss: 0.5974 - val_loss: 1.3280
Epoch 4/10
23/23 [==============================] - 5s 39ms/step
### Start fold 5


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
92/92 [==============================] - 30s 144ms/step - loss: 0.9745 - val_loss: 0.5757
Epoch 2/10
92/92 [==============================] - 10s 107ms/step - loss: 0.6831 - val_loss: 0.5270
Epoch 3/10
92/92 [==============================] - 9s 103ms/step - loss: 0.5439 - val_loss: 0.6266
Epoch 4/10
23/23 [==============================] - 5s 41ms/step


   Dataset    Model Distant      Loss  Accuracy  Precision    Recall  \
0      sg1  roberta   False  0.580312  0.515538   0.562240  0.636251   
1      sg2  roberta   False  0.512574  0.545579   0.522718  0.595028   
2      sg1  deberta   False  1.786612  0.513528   0.431187  0.633163   
3      sg2  deberta   False  2.062078  0.500138   0.296457  0.600000   
4      sg1  electra   False  0.550895  0.475357   0.444049  0.380805   
5      sg2  electra   False  0.482666  0.506686   0.651247  0.436464   
6      sg1  deberta    True  0.617424  0.492855   0.334588  0.584009   
7      sg2  deberta    True  0.620772  0.412249   0.402475  0.313812   
8      sg1     bert    True  1.253572  0.452740   0.246955  0.457718   
9      sg2     bert    True  0.532223  0.558989   0.539072  0.743646   
10     sg1  roberta    True  0.546565  0.468920   0.549649  0.386568   
11     sg2  roberta    True  0.498680  0.570941   0.598381  0.611602   

          F1  F1 Micro  F1 Weighted  
0   0.460742  0.515538   

# DeBERTa w/ Distant

In [ ]:
# Sg1 
results = measure(run_model_5fold('sg1', df_sg1, 'deberta', freeze_encoder=False, pretrained=True, plot=False), results)
print(results)

All model checkpoint layers were used when initializing TFDebertaForSequenceClassification.

Some layers of TFDebertaForSequenceClassification were not initialized from the model checkpoint at kamalkraj/deberta-base and are newly initialized: ['classifier', 'cls_dropout', 'pooler']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Start fold 1


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
All model checkpoint layers were used when initializing TFDebertaForSequenceClassification.

Some layers of TFDebertaForSequenceClassification were not initialized from the model checkpoint at kamalkraj/deberta-base and are newly initialized: ['classifier', 'cls_dropout', 'pooler']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
39/39 [==============================] - 55s 505ms/step - loss: 0.7873 - val_loss: 0.6543
Epoch 2/10
39/39 [==============================] - 12s 320ms/step - loss: 0.7209 - val_loss: 0.7050
Epoch 3/10
10/10 [==============================] - 7s 45ms/step
### Start fold 2


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
All model checkpoint layers were used when initializing TFDebertaForSequenceClassification.

Some layers of TFDebertaForSequenceClassification were not initialized from the model checkpoint at kamalkraj/deberta-base and are newly initialized: ['classifier', 'cls_dropout', 'pooler']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
All model checkpoint layers were used when initializing TFDebertaForSequenceClassification.

Some layers of TFDebertaForSequenceClassification were not initialized from the model checkpoint at kamalkraj/deberta-base and are newly initialized: ['classifier', 'cls_dropout', 'pooler']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
39/39 [==============================] - 56s 509ms/step - loss: 1.6339 - val_loss: 0.6588
Epoch 2/10
39/39 [==============================] - 12s 319ms/step - loss: 0.6686 - val_loss: 0.6659
Epoch 3/10
39/39 [==============================] - 13s 330ms/step - loss: 0.6466 - val_loss: 0.6027
Epoch 4/10
39/39 [==============================] - 13s 328ms/step - loss: 0.5569 - val_loss: 0.5793
Epoch 5/10
39/39 [==============================] - 12s 319ms/step - loss: 0.6418 - val_loss: 0.6071
Epoch 6/10
10/10 [==============================] - 7s 42ms/step
### Start fold 5


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
All model checkpoint layers were used when initializing TFDebertaForSequenceClassification.

Some layers of TFDebertaForSequenceClassification were not initialized from the model checkpoint at kamalkraj/deberta-base and are newly initialized: ['classifier', 'cls_dropout', 'pooler']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
39/39 [==============================] - 56s 504ms/step - loss: 0.7632 - val_loss: 0.5815
Epoch 2/10
39/39 [==============================] - 13s 324ms/step - loss: 0.5804 - val_loss: 0.5169
Epoch 3/10
39/39 [==============================] - 12s 316ms/step - loss: 0.6132 - val_loss: 1.0005
Epoch 4/10
10/10 [==============================] - 7s 54ms/step


  Dataset    Model Distant      Loss  Accuracy  Precision    Recall        F1  \
0     sg1  roberta   False  0.580312  0.515538   0.562240  0.636251  0.460742   
1     sg2  roberta   False  0.512574  0.545579   0.522718  0.595028  0.418805   
2     sg1  deberta   False  1.786612  0.513528   0.431187  0.633163  0.467586   
3     sg2  deberta   False  2.062078  0.500138   0.296457  0.600000  0.396837   
4     sg1  electra   False  0.550895  0.475357   0.444049  0.380805  0.393093   
5     sg2  electra   False  0.482666  0.506686   0.651247  0.436464  0.394047   
6     sg1  deberta    True  0.617424  0.492855   0.334588  0.584009  0.418621   

   F1 Micro  F1 Weighted  
0  0.515538     0.377869  
1  0.545579     0.419941  
2  0.513528     0.389934  
3  0.500138     0.335278  
4  0.475357     0.448213  
5  0.506686     0.440985  
6  0.492855     0.396586  


In [ ]:
# Sg2
results = measure(run_model_5fold('sg2', df_sg2, 'deberta', freeze_encoder=False, pretrained=True, plot=False), results)
print(results)

All model checkpoint layers were used when initializing TFDebertaForSequenceClassification.

Some layers of TFDebertaForSequenceClassification were not initialized from the model checkpoint at kamalkraj/deberta-base and are newly initialized: ['classifier', 'cls_dropout', 'pooler']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Start fold 1


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
All model checkpoint layers were used when initializing TFDebertaForSequenceClassification.

Some layers of TFDebertaForSequenceClassification were not initialized from the model checkpoint at kamalkraj/deberta-base and are newly initialized: ['classifier', 'cls_dropout', 'pooler']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
92/92 [==============================] - 72s 398ms/step - loss: 0.7597 - val_loss: 0.6905
Epoch 2/10
92/92 [==============================] - 30s 325ms/step - loss: 0.5920 - val_loss: 0.5679
Epoch 3/10
92/92 [==============================] - 30s 322ms/step - loss: 0.5050 - val_loss: 0.7358
Epoch 4/10
23/23 [==============================] - 7s 48ms/step
### Start fold 2


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
All model checkpoint layers were used when initializing TFDebertaForSequenceClassification.

Some layers of TFDebertaForSequenceClassification were not initialized from the model checkpoint at kamalkraj/deberta-base and are newly initialized: ['classifier', 'cls_dropout', 'pooler']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Epoch 1/10
92/92 [==============================] - 74s 402ms/step - loss: 1.5932 - val_loss: 0.7086
Epoch 2/10
92/92 [==============================] - 30s 321ms/step - loss: 0.7898 - val_loss: 1.1221
Epoch 3/10
23/23 [==============================] - 7s 47ms/step
### Start fold 5


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
All model checkpoint layers were used when initializing TFDebertaForSequenceClassification.

Some layers of TFDebertaForSequenceClassification were not initialized from the model checkpoint at kamalkraj/deberta-base and are newly initialized: ['classifier', 'cls_dropout', 'pooler']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
92/92 [==============================] - 70s 359ms/step - loss: 0.8211 - val_loss: 0.6885
Epoch 2/10
92/92 [==============================] - 27s 290ms/step - loss: 0.7076 - val_loss: 0.6619
Epoch 3/10
92/92 [==============================] - 26s 285ms/step - loss: 0.5853 - val_loss: 0.7247
Epoch 4/10
23/23 [==============================] - 8s 55ms/step


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


  Dataset    Model Distant      Loss  Accuracy  Precision    Recall        F1  \
0     sg1  roberta   False  0.580312  0.515538   0.562240  0.636251  0.460742   
1     sg2  roberta   False  0.512574  0.545579   0.522718  0.595028  0.418805   
2     sg1  deberta   False  1.786612  0.513528   0.431187  0.633163  0.467586   
3     sg2  deberta   False  2.062078  0.500138   0.296457  0.600000  0.396837   
4     sg1  electra   False  0.550895  0.475357   0.444049  0.380805  0.393093   
5     sg2  electra   False  0.482666  0.506686   0.651247  0.436464  0.394047   
6     sg1  deberta    True  0.617424  0.492855   0.334588  0.584009  0.418621   
7     sg2  deberta    True  0.620772  0.412249   0.402475  0.313812  0.241562   

   F1 Micro  F1 Weighted  
0  0.515538     0.377869  
1  0.545579     0.419941  
2  0.513528     0.389934  
3  0.500138     0.335278  
4  0.475357     0.448213  
5  0.506686     0.440985  
6  0.492855     0.396586  
7  0.412249     0.310578  


# ELECTRA

In [ ]:
# Sg1 
results = measure(run_model_5fold('sg1', df_sg1, 'electra', freeze_encoder=False, pretrained=False, plot=False), results)
print(results)

### Start fold 1


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/54.5M [00:00<?, ?B/s]

...

Epoch 1/10
39/39 [==============================] - 23s 161ms/step - loss: 0.8942 - val_loss: 0.6923
Epoch 2/10
39/39 [==============================] - 2s 61ms/step - loss: 0.6892 - val_loss: 0.6843
Epoch 3/10
39/39 [==============================] - 2s 61ms/step - loss: 0.6553 - val_loss: 0.6302
Epoch 4/10
39/39 [==============================] - 2s 61ms/step - loss: 0.5722 - val_loss: 0.6028
Epoch 5/10
39/39 [==============================] - 2s 58ms/step - loss: 0.4381 - val_loss: 0.6169
Epoch 6/10
10/10 [==============================] - 3s 25ms/step
### Start fold 4


Some layers from the model checkpoint at google/electra-small-discriminator were not used when initializing TFElectraForSequenceClassification: ['discriminator_predictions']
- This IS expected if you are initializing TFElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
39/39 [==============================] - 23s 160ms/step - loss: 1.0848 - val_loss: 0.7142
Epoch 2/10
39/39 [==============================] - 2s 61ms/step - loss: 0.7022 - val_loss: 0.6896
Epoch 3/10
39/39 [==============================] - 2s 60ms/step - loss: 0.6880 - val_loss: 0.6887
Epoch 4/10
39/39 [==============================] - 2s 61ms/step - loss: 0.6244 - val_loss: 0.5915
Epoch 5/10
39/39 [==============================] - 2s 59ms/step - loss: 0.5295 - val_loss: 0.7560
Epoch 6/10
10/10 [==============================] - 3s 26ms/step
### Start fold 5


Some layers from the model checkpoint at google/electra-small-discriminator were not used when initializing TFElectraForSequenceClassification: ['discriminator_predictions']
- This IS expected if you are initializing TFElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
39/39 [==============================] - 23s 162ms/step - loss: 1.9510 - val_loss: 0.7534
Epoch 2/10
39/39 [==============================] - 2s 62ms/step - loss: 0.7208 - val_loss: 0.6937
Epoch 3/10
39/39 [==============================] - 2s 61ms/step - loss: 0.6942 - val_loss: 0.6861
Epoch 4/10
39/39 [==============================] - 2s 61ms/step - loss: 0.6853 - val_loss: 0.6661
Epoch 5/10
39/39 [==============================] - 2s 61ms/step - loss: 0.6066 - val_loss: 0.6025
Epoch 6/10
39/39 [==============================] - 2s 59ms/step - loss: 0.5294 - val_loss: 1.0345
Epoch 7/10
39/39 [==============================] - 2s 61ms/step - loss: 0.4539 - val_loss: 0.5701
Epoch 8/10
39/39 [==============================] - 2s 59ms/step - loss: 0.3396 - val_loss: 1.8429
Epoch 9/10
10/10 [==============================] - 3s 27ms/step
  Dataset    Model Distant      Loss  Accuracy  Precision    Recall        F1  \
0     sg1  roberta   False  0.580312  0.515538   0.562240  0

In [ ]:
# Sg2 
results = measure(run_model_5fold('sg2', df_sg2, 'electra', freeze_encoder=False, pretrained=False, plot=False), results)
print(results)

### Start fold 1


Some layers from the model checkpoint at google/electra-small-discriminator were not used when initializing TFElectraForSequenceClassification: ['discriminator_predictions']
- This IS expected if you are initializing TFElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
92/92 [==============================] - 25s 93ms/step - loss: 0.8252 - val_loss: 0.6461
Epoch 2/10
92/92 [==============================] - 5s 59ms/step - loss: 0.5869 - val_loss: 0.5166
Epoch 3/10
92/92 [==============================] - 5s 58ms/step - loss: 0.5222 - val_loss: 0.5501
Epoch 4/10
23/23 [==============================] - 4s 26ms/step
### Start fold 2


Some layers from the model checkpoint at google/electra-small-discriminator were not used when initializing TFElectraForSequenceClassification: ['discriminator_predictions']
- This IS expected if you are initializing TFElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
92/92 [==============================] - 26s 102ms/step - loss: 0.8490 - val_loss: 0.6674
Epoch 2/10
92/92 [==============================] - 6s 60ms/step - loss: 0.5922 - val_loss: 0.4950
Epoch 3/10
92/92 [==============================] - 5s 59ms/step - loss: 0.4859 - val_loss: 0.4766
Epoch 4/10
92/92 [==============================] - 5s 58ms/step - loss: 0.4076 - val_loss: 0.5213
Epoch 5/10
23/23 [==============================] - 4s 26ms/step
### Start fold 3


...

Epoch 1/10
92/92 [==============================] - 26s 101ms/step - loss: 0.8817 - val_loss: 0.6276
Epoch 2/10
92/92 [==============================] - 5s 60ms/step - loss: 0.5570 - val_loss: 0.4823
Epoch 3/10
92/92 [==============================] - 5s 59ms/step - loss: 0.4527 - val_loss: 0.4667
Epoch 4/10
92/92 [==============================] - 5s 59ms/step - loss: 0.3757 - val_loss: 0.4552
Epoch 5/10
92/92 [==============================] - 5s 59ms/step - loss: 0.3480 - val_loss: 0.9295
Epoch 6/10
23/23 [==============================] - 4s 26ms/step
### Start fold 4


Some layers from the model checkpoint at google/electra-small-discriminator were not used when initializing TFElectraForSequenceClassification: ['discriminator_predictions']
- This IS expected if you are initializing TFElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
92/92 [==============================] - 26s 102ms/step - loss: 0.8090 - val_loss: 0.6038
Epoch 2/10
92/92 [==============================] - 5s 60ms/step - loss: 0.5422 - val_loss: 0.5121
Epoch 3/10
92/92 [==============================] - 5s 59ms/step - loss: 0.4488 - val_loss: 0.5146
Epoch 4/10
23/23 [==============================] - 4s 26ms/step
### Start fold 5


Some layers from the model checkpoint at google/electra-small-discriminator were not used when initializing TFElectraForSequenceClassification: ['discriminator_predictions']
- This IS expected if you are initializing TFElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
92/92 [==============================] - 25s 99ms/step - loss: 0.8364 - val_loss: 0.6593
Epoch 2/10
92/92 [==============================] - 5s 57ms/step - loss: 0.6018 - val_loss: 0.5877
Epoch 3/10
92/92 [==============================] - 5s 57ms/step - loss: 0.4835 - val_loss: 0.4560
Epoch 4/10
92/92 [==============================] - 5s 56ms/step - loss: 0.5616 - val_loss: 0.5526
Epoch 5/10
92/92 [==============================] - 5s 58ms/step - loss: 0.4302 - val_loss: 0.4528
Epoch 6/10
92/92 [==============================] - 5s 56ms/step - loss: 0.3089 - val_loss: 0.5342
Epoch 7/10
23/23 [==============================] - 4s 27ms/step
  Dataset    Model Distant      Loss  Accuracy  Precision    Recall        F1  \
0     sg1  roberta   False  0.580312  0.515538   0.562240  0.636251  0.460742   
1     sg2  roberta   False  0.512574  0.545579   0.522718  0.595028  0.418805   
2     sg1  deberta   False  1.786612  0.513528   0.431187  0.633163  0.467586   
3     sg2  deber

# Analysis

In [34]:
df_sg1.head()

,sentence,news_link,outlet,topic,type,Label_bias,label_opinion,biased_words
0,The Republican president assumed he was helpin...,http://www.msnbc.com/rachel-maddow-show/auto-i...,msnbc,environment,left,1,Expresses writer’s opinion,[]
1,Though the indictment of a woman for her own p...,https://eu.usatoday.com/story/news/nation/2019...,usa-today,abortion,center,0,Somewhat factual but also opinionated,[]
3,The tragedy of America’s 18 years in Afghanist...,http://feedproxy.google.com/~r/breitbart/~3/ER...,breitbart,international-politics-and-world-news,right,1,Somewhat factual but also opinionated,"['tragedy', 'stubborn']"
4,The justices threw out a challenge from gun ri...,https://www.huffpost.com/entry/supreme-court-g...,msnbc,gun-control,left,0,Entirely factual,[]
5,A review of his posts in online message boards...,https://eu.usatoday.com/story/news/nation/2020...,usa-today,white-nationalism,center,1,Entirely factual,['plant']


In [10]:
df_sg2.head()

,sentence,news_link,outlet,topic,type,Label_bias,label_opinion,biased_words
0,"""Orange Is the New Black"" star Yael Stone is r...",https://www.foxnews.com/entertainment/australi...,Fox News,environment,right,0,Entirely factual,[]
1,"""We have one beautiful law,"" Trump recently sa...",https://www.alternet.org/2020/06/law-and-order...,Alternet,gun control,left,1,Somewhat factual but also opinionated,"['bizarre', 'characteristically']"
2,"...immigrants as criminals and eugenics, all o...",https://www.nbcnews.com/news/latino/after-step...,MSNBC,white-nationalism,left,1,Expresses writer’s opinion,"['criminals', 'fringe', 'extreme']"
3,...we sounded the alarm in the early months of...,https://www.alternet.org/2019/07/fox-news-has-...,Alternet,white-nationalism,left,1,Somewhat factual but also opinionated,[]
4,[Black Lives Matter] is essentially a non-fals...,http://feedproxy.google.com/~r/breitbart/~3/-v...,Breitbart,marriage-equality,NaN,1,Expresses writer’s opinion,['cult']


In [52]:
# sg1 sentences
fact_bias = df_sg1.loc[(df_sg1['label_opinion']=='Entirely factual') & (df_sg1['Label_bias']==1)].sample().iloc[0]
fact_nonbias = df_sg1.loc[(df_sg1['label_opinion']=='Entirely factual') & (df_sg1['Label_bias']==0)].sample().iloc[0]
opin_bias = df_sg1.loc[(df_sg1['label_opinion']=='Expresses writer’s opinion') & (df_sg1['Label_bias']==1)].sample().iloc[0]
opin_nonbias = df_sg1.loc[(df_sg1['label_opinion']=='Expresses writer’s opinion') & (df_sg1['Label_bias']==0)].sample().iloc[0]
both_bias = df_sg1.loc[(df_sg1['label_opinion']=='Somewhat factual but also opinionated') & (df_sg1['Label_bias']==1)].sample().iloc[0]
both_nonbias = df_sg1.loc[(df_sg1['label_opinion']=='Somewhat factual but also opinionated') & (df_sg1['Label_bias']==0)].sample().iloc[0]

sentences = [fact_bias, fact_nonbias, opin_bias, opin_nonbias, both_bias, both_nonbias]

In [ ]:
# BERT SG1
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5) 
model.compile(optimizer=optimizer, loss='binary_crossentropy') 
model.load_weights(f'/content/drive/MyDrive/Colab Notebooks/weights/bert_False_sg1_main')
trained_model_layer = model.get_layer(index=0).get_weights()
model.get_layer(index=0).set_weights(trained_model_layer)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [54]:
cols = ['Opinon Label', 'Sentence', 'Predicted', 'Actual']
output = pd.DataFrame(columns=cols)

for s in sentences:
  inputs = tokenizer(s['sentence'], return_tensors="tf")
  logits = model(**inputs).logits
  predicted_class_id = int(tf.math.argmax(logits, axis=-1)[0])
  row = {'Opinon Label': s['label_opinion'], 
         'Sentence': s['sentence'], 
         'Predicted': predicted_class_id, 
         'Actual': s['Label_bias']
    }
    
  output = output.append(row, ignore_index=True)

print(output)

                            Opinon Label  \
0                       Entirely factual   
1                       Entirely factual   
2             Expresses writer’s opinion   
3             Expresses writer’s opinion   
4  Somewhat factual but also opinionated   
5  Somewhat factual but also opinionated   

                                            Sentence Predicted Actual  
0  While sexual ambiguity does occasionally occur...         0      1  
1  George Washington University (GW)’s Parliament...         0      0  
2  Perhaps that is why both Warren and Sanders ha...         1      1  
3  Despite a pandemic, one-sided presidential nom...         1      0  
4  Before President Donald Trump threatened to vi...         0      1  
5  At the dawn of the year 2010, few Americans co...         0      0  


In [ ]:
# BERT Distant SG1
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5) 
model.compile(optimizer=optimizer, loss='binary_crossentropy') 
model.load_weights(f'/content/drive/MyDrive/Colab Notebooks/weights/bert_True_sg1_main')
trained_model_layer = model.get_layer(index=0).get_weights()
model.get_layer(index=0).set_weights(trained_model_layer)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

gc.collect()

In [56]:
cols = ['Opinon Label', 'Sentence', 'Predicted', 'Actual']
output = pd.DataFrame(columns=cols)

for s in sentences:
  inputs = tokenizer(s['sentence'], return_tensors="tf")
  logits = model(**inputs).logits
  predicted_class_id = int(tf.math.argmax(logits, axis=-1)[0])
  row = {'Opinon Label': s['label_opinion'], 
         'Sentence': s['sentence'], 
         'Predicted': predicted_class_id, 
         'Actual': s['Label_bias']
    }
    
  output = output.append(row, ignore_index=True)

print(output)

                            Opinon Label  \
0                       Entirely factual   
1                       Entirely factual   
2             Expresses writer’s opinion   
3             Expresses writer’s opinion   
4  Somewhat factual but also opinionated   
5  Somewhat factual but also opinionated   

                                            Sentence Predicted Actual  
0  While sexual ambiguity does occasionally occur...         1      1  
1  George Washington University (GW)’s Parliament...         0      0  
2  Perhaps that is why both Warren and Sanders ha...         1      1  
3  Despite a pandemic, one-sided presidential nom...         0      0  
4  Before President Donald Trump threatened to vi...         1      1  
5  At the dawn of the year 2010, few Americans co...         0      0  


In [ ]:
# RoBERTa Distant SG1
model = TFBertForSequenceClassification.from_pretrained('roberta-base')
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5) 
model.compile(optimizer=optimizer, loss='binary_crossentropy') 
model.load_weights(f'/content/drive/MyDrive/Colab Notebooks/weights/roberta_True_sg1_main')
trained_model_layer = model.get_layer(index=0).get_weights()
model.get_layer(index=0).set_weights(trained_model_layer)

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [58]:
cols = ['Opinon Label', 'Sentence', 'Predicted', 'Actual']
output = pd.DataFrame(columns=cols)

for s in sentences:
  inputs = tokenizer(s['sentence'], return_tensors="tf")
  logits = model(**inputs).logits
  predicted_class_id = int(tf.math.argmax(logits, axis=-1)[0])
  row = {'Opinon Label': s['label_opinion'], 
         'Sentence': s['sentence'], 
         'Predicted': predicted_class_id, 
         'Actual': s['Label_bias']
    }
    
  output = output.append(row, ignore_index=True)

print(output)

                            Opinon Label  \
0                       Entirely factual   
1                       Entirely factual   
2             Expresses writer’s opinion   
3             Expresses writer’s opinion   
4  Somewhat factual but also opinionated   
5  Somewhat factual but also opinionated   

                                            Sentence Predicted Actual  
0  While sexual ambiguity does occasionally occur...         0      1  
1  George Washington University (GW)’s Parliament...         1      0  
2  Perhaps that is why both Warren and Sanders ha...         0      1  
3  Despite a pandemic, one-sided presidential nom...         0      0  
4  Before President Donald Trump threatened to vi...         1      1  
5  At the dawn of the year 2010, few Americans co...         0      0  


In [ ]:
# DeBERTa SG1
model = TFBertForSequenceClassification.from_pretrained("kamalkraj/deberta-base")
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5) 
model.compile(optimizer=optimizer, loss='binary_crossentropy') 
model.load_weights(f'/content/drive/MyDrive/Colab Notebooks/weights/deberta_False_sg1_main')
trained_model_layer = model.get_layer(index=0).get_weights()
model.get_layer(index=0).set_weights(trained_model_layer)

tokenizer = DebertaTokenizer.from_pretrained("kamalkraj/deberta-base")

gc.collect()

In [60]:
cols = ['Opinon Label', 'Sentence', 'Predicted', 'Actual']
output = pd.DataFrame(columns=cols)

for s in sentences:
  inputs = tokenizer(s['sentence'], return_tensors="tf")
  logits = model(**inputs).logits
  predicted_class_id = int(tf.math.argmax(logits, axis=-1)[0])
  row = {'Opinon Label': s['label_opinion'], 
         'Sentence': s['sentence'], 
         'Predicted': predicted_class_id, 
         'Actual': s['Label_bias']
    }
    
  output = output.append(row, ignore_index=True)

print(output)

                            Opinon Label  \
0                       Entirely factual   
1                       Entirely factual   
2             Expresses writer’s opinion   
3             Expresses writer’s opinion   
4  Somewhat factual but also opinionated   
5  Somewhat factual but also opinionated   

                                            Sentence Predicted Actual  
0  While sexual ambiguity does occasionally occur...         0      1  
1  George Washington University (GW)’s Parliament...         1      0  
2  Perhaps that is why both Warren and Sanders ha...         0      1  
3  Despite a pandemic, one-sided presidential nom...         0      0  
4  Before President Donald Trump threatened to vi...         0      1  
5  At the dawn of the year 2010, few Americans co...         0      0  


In [ ]:
# DeBERTa Distant SG1
model = TFBertForSequenceClassification.from_pretrained("kamalkraj/deberta-base")
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5) 
model.compile(optimizer=optimizer, loss='binary_crossentropy') 
model.load_weights(f'/content/drive/MyDrive/Colab Notebooks/weights/deberta_True_sg1_main')
trained_model_layer = model.get_layer(index=0).get_weights()
model.get_layer(index=0).set_weights(trained_model_layer)

tokenizer = DebertaTokenizer.from_pretrained("kamalkraj/deberta-base")

gc.collect()

In [62]:
cols = ['Opinon Label', 'Sentence', 'Predicted', 'Actual']
output = pd.DataFrame(columns=cols)

for s in sentences:
  inputs = tokenizer(s['sentence'], return_tensors="tf")
  logits = model(**inputs).logits
  predicted_class_id = int(tf.math.argmax(logits, axis=-1)[0])
  row = {'Opinon Label': s['label_opinion'], 
         'Sentence': s['sentence'], 
         'Predicted': predicted_class_id, 
         'Actual': s['Label_bias']
    }
    
  output = output.append(row, ignore_index=True)

print(output)

                            Opinon Label  \
0                       Entirely factual   
1                       Entirely factual   
2             Expresses writer’s opinion   
3             Expresses writer’s opinion   
4  Somewhat factual but also opinionated   
5  Somewhat factual but also opinionated   

                                            Sentence Predicted Actual  
0  While sexual ambiguity does occasionally occur...         0      1  
1  George Washington University (GW)’s Parliament...         0      0  
2  Perhaps that is why both Warren and Sanders ha...         0      1  
3  Despite a pandemic, one-sided presidential nom...         0      0  
4  Before President Donald Trump threatened to vi...         0      1  
5  At the dawn of the year 2010, few Americans co...         0      0  


In [ ]:
# ELECTRA SG1
model = TFBertForSequenceClassification.from_pretrained('google/electra-small-discriminator')
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5) 
model.compile(optimizer=optimizer, loss='binary_crossentropy') 
model.load_weights(f'/content/drive/MyDrive/Colab Notebooks/weights/electra_False_sg1_main')
trained_model_layer = model.get_layer(index=0).get_weights()
model.get_layer(index=0).set_weights(trained_model_layer)

tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-discriminator')

gc.collect()

In [64]:
cols = ['Opinon Label', 'Sentence', 'Predicted', 'Actual']
output = pd.DataFrame(columns=cols)

for s in sentences:
  inputs = tokenizer(s['sentence'], return_tensors="tf")
  logits = model(**inputs).logits
  predicted_class_id = int(tf.math.argmax(logits, axis=-1)[0])
  row = {'Opinon Label': s['label_opinion'], 
         'Sentence': s['sentence'], 
         'Predicted': predicted_class_id, 
         'Actual': s['Label_bias']
    }
    
  output = output.append(row, ignore_index=True)

print(output)

                            Opinon Label  \
0                       Entirely factual   
1                       Entirely factual   
2             Expresses writer’s opinion   
3             Expresses writer’s opinion   
4  Somewhat factual but also opinionated   
5  Somewhat factual but also opinionated   

                                            Sentence Predicted Actual  
0  While sexual ambiguity does occasionally occur...         1      1  
1  George Washington University (GW)’s Parliament...         0      0  
2  Perhaps that is why both Warren and Sanders ha...         1      1  
3  Despite a pandemic, one-sided presidential nom...         1      0  
4  Before President Donald Trump threatened to vi...         1      1  
5  At the dawn of the year 2010, few Americans co...         0      0  


In [65]:
# sg2 sentences
fact_bias = df_sg2.loc[(df_sg2['label_opinion']=='Entirely factual') & (df_sg2['Label_bias']==1)].sample().iloc[0]
fact_nonbias = df_sg2.loc[(df_sg2['label_opinion']=='Entirely factual') & (df_sg2['Label_bias']==0)].sample().iloc[0]
opin_bias = df_sg2.loc[(df_sg2['label_opinion']=='Expresses writer’s opinion') & (df_sg2['Label_bias']==1)].sample().iloc[0]
opin_nonbias = df_sg2.loc[(df_sg2['label_opinion']=='Expresses writer’s opinion') & (df_sg2['Label_bias']==0)].sample().iloc[0]
both_bias = df_sg2.loc[(df_sg2['label_opinion']=='Somewhat factual but also opinionated') & (df_sg2['Label_bias']==1)].sample().iloc[0]
both_nonbias = df_sg2.loc[(df_sg2['label_opinion']=='Somewhat factual but also opinionated') & (df_sg2['Label_bias']==0)].sample().iloc[0]

sentences = [fact_bias, fact_nonbias, opin_bias, opin_nonbias, both_bias, both_nonbias]

In [ ]:
# BERT Distant SG2
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5) 
model.compile(optimizer=optimizer, loss='binary_crossentropy') 
model.load_weights(f'/content/drive/MyDrive/Colab Notebooks/weights/bert_True_sg2_main')
trained_model_layer = model.get_layer(index=0).get_weights()
model.get_layer(index=0).set_weights(trained_model_layer)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

gc.collect()

In [67]:
cols = ['Opinon Label', 'Sentence', 'Predicted', 'Actual']
output = pd.DataFrame(columns=cols)

for s in sentences:
  inputs = tokenizer(s['sentence'], return_tensors="tf")
  logits = model(**inputs).logits
  predicted_class_id = int(tf.math.argmax(logits, axis=-1)[0])
  row = {'Opinon Label': s['label_opinion'], 
         'Sentence': s['sentence'], 
         'Predicted': predicted_class_id, 
         'Actual': s['Label_bias']
    }
    
  output = output.append(row, ignore_index=True)

print(output)

                            Opinon Label  \
0                       Entirely factual   
1                       Entirely factual   
2             Expresses writer’s opinion   
3             Expresses writer’s opinion   
4  Somewhat factual but also opinionated   
5  Somewhat factual but also opinionated   

                                            Sentence Predicted Actual  
0  A tear slipped down the cheek of Ghanaian-Germ...         1      1  
1  Swedish police say they are searching for a bl...         1      0  
2  The Doctor Who franchise has tested this notio...         1      1  
3  Reducing the upper tax rate can often, counter...         0      0  
4  While the mayor has severely curtailed religio...         0      1  
5  Although Facebook Chief Operating Officer Sher...         1      0  


In [ ]:
# RoBERTa Distant SG2
model = TFBertForSequenceClassification.from_pretrained('roberta-base')
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5) 
model.compile(optimizer=optimizer, loss='binary_crossentropy') 
model.load_weights(f'/content/drive/MyDrive/Colab Notebooks/weights/roberta_True_sg2_main')
trained_model_layer = model.get_layer(index=0).get_weights()
model.get_layer(index=0).set_weights(trained_model_layer)

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

gc.collect()

In [69]:
cols = ['Opinon Label', 'Sentence', 'Predicted', 'Actual']
output = pd.DataFrame(columns=cols)

for s in sentences:
  inputs = tokenizer(s['sentence'], return_tensors="tf")
  logits = model(**inputs).logits
  predicted_class_id = int(tf.math.argmax(logits, axis=-1)[0])
  row = {'Opinon Label': s['label_opinion'], 
         'Sentence': s['sentence'], 
         'Predicted': predicted_class_id, 
         'Actual': s['Label_bias']
    }
    
  output = output.append(row, ignore_index=True)

print(output)

                            Opinon Label  \
0                       Entirely factual   
1                       Entirely factual   
2             Expresses writer’s opinion   
3             Expresses writer’s opinion   
4  Somewhat factual but also opinionated   
5  Somewhat factual but also opinionated   

                                            Sentence Predicted Actual  
0  A tear slipped down the cheek of Ghanaian-Germ...         0      1  
1  Swedish police say they are searching for a bl...         0      0  
2  The Doctor Who franchise has tested this notio...         1      1  
3  Reducing the upper tax rate can often, counter...         0      0  
4  While the mayor has severely curtailed religio...         0      1  
5  Although Facebook Chief Operating Officer Sher...         0      0  


In [ ]:
# DeBERTa SG2
model = TFBertForSequenceClassification.from_pretrained("kamalkraj/deberta-base")
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5) 
model.compile(optimizer=optimizer, loss='binary_crossentropy') 
model.load_weights(f'/content/drive/MyDrive/Colab Notebooks/weights/deberta_False_sg2_main')
trained_model_layer = model.get_layer(index=0).get_weights()
model.get_layer(index=0).set_weights(trained_model_layer)

tokenizer = DebertaTokenizer.from_pretrained("kamalkraj/deberta-base")

gc.collect()

In [71]:
cols = ['Opinon Label', 'Sentence', 'Predicted', 'Actual']
output = pd.DataFrame(columns=cols)

for s in sentences:
  inputs = tokenizer(s['sentence'], return_tensors="tf")
  logits = model(**inputs).logits
  predicted_class_id = int(tf.math.argmax(logits, axis=-1)[0])
  row = {'Opinon Label': s['label_opinion'], 
         'Sentence': s['sentence'], 
         'Predicted': predicted_class_id, 
         'Actual': s['Label_bias']
    }
    
  output = output.append(row, ignore_index=True)

print(output)

                            Opinon Label  \
0                       Entirely factual   
1                       Entirely factual   
2             Expresses writer’s opinion   
3             Expresses writer’s opinion   
4  Somewhat factual but also opinionated   
5  Somewhat factual but also opinionated   

                                            Sentence Predicted Actual  
0  A tear slipped down the cheek of Ghanaian-Germ...         0      1  
1  Swedish police say they are searching for a bl...         0      0  
2  The Doctor Who franchise has tested this notio...         0      1  
3  Reducing the upper tax rate can often, counter...         0      0  
4  While the mayor has severely curtailed religio...         0      1  
5  Although Facebook Chief Operating Officer Sher...         0      0  


In [ ]:
# DeBERTa Distant SG2
model = TFBertForSequenceClassification.from_pretrained("kamalkraj/deberta-base")
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5) 
model.compile(optimizer=optimizer, loss='binary_crossentropy') 
model.load_weights(f'/content/drive/MyDrive/Colab Notebooks/weights/deberta_True_sg2_main')
trained_model_layer = model.get_layer(index=0).get_weights()
model.get_layer(index=0).set_weights(trained_model_layer)

tokenizer = DebertaTokenizer.from_pretrained("kamalkraj/deberta-base")

gc.collect()

In [73]:
cols = ['Opinon Label', 'Sentence', 'Predicted', 'Actual']
output = pd.DataFrame(columns=cols)

for s in sentences:
  inputs = tokenizer(s['sentence'], return_tensors="tf")
  logits = model(**inputs).logits
  predicted_class_id = int(tf.math.argmax(logits, axis=-1)[0])
  row = {'Opinon Label': s['label_opinion'], 
         'Sentence': s['sentence'], 
         'Predicted': predicted_class_id, 
         'Actual': s['Label_bias']
    }
    
  output = output.append(row, ignore_index=True)

print(output)

                            Opinon Label  \
0                       Entirely factual   
1                       Entirely factual   
2             Expresses writer’s opinion   
3             Expresses writer’s opinion   
4  Somewhat factual but also opinionated   
5  Somewhat factual but also opinionated   

                                            Sentence Predicted Actual  
0  A tear slipped down the cheek of Ghanaian-Germ...         1      1  
1  Swedish police say they are searching for a bl...         1      0  
2  The Doctor Who franchise has tested this notio...         1      1  
3  Reducing the upper tax rate can often, counter...         1      0  
4  While the mayor has severely curtailed religio...         1      1  
5  Although Facebook Chief Operating Officer Sher...         1      0  


In [ ]:
# ELECTRA SG2
model = TFBertForSequenceClassification.from_pretrained('google/electra-small-discriminator')
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5) 
model.compile(optimizer=optimizer, loss='binary_crossentropy') 
model.load_weights(f'/content/drive/MyDrive/Colab Notebooks/weights/electra_False_sg2_main')
trained_model_layer = model.get_layer(index=0).get_weights()
model.get_layer(index=0).set_weights(trained_model_layer)

tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-discriminator')

gc.collect()

In [75]:
cols = ['Opinon Label', 'Sentence', 'Predicted', 'Actual']
output = pd.DataFrame(columns=cols)

for s in sentences:
  inputs = tokenizer(s['sentence'], return_tensors="tf")
  logits = model(**inputs).logits
  predicted_class_id = int(tf.math.argmax(logits, axis=-1)[0])
  row = {'Opinon Label': s['label_opinion'], 
         'Sentence': s['sentence'], 
         'Predicted': predicted_class_id, 
         'Actual': s['Label_bias']
    }
    
  output = output.append(row, ignore_index=True)

print(output)

                            Opinon Label  \
0                       Entirely factual   
1                       Entirely factual   
2             Expresses writer’s opinion   
3             Expresses writer’s opinion   
4  Somewhat factual but also opinionated   
5  Somewhat factual but also opinionated   

                                            Sentence Predicted Actual  
0  A tear slipped down the cheek of Ghanaian-Germ...         0      1  
1  Swedish police say they are searching for a bl...         0      0  
2  The Doctor Who franchise has tested this notio...         1      1  
3  Reducing the upper tax rate can often, counter...         0      0  
4  While the mayor has severely curtailed religio...         0      1  
5  Although Facebook Chief Operating Officer Sher...         0      0  
